In [1]:
import tensorflow as tf
import numpy as np
from model import *
import glob
from video_loader import DeepFakeTransformer
import pathlib

In [2]:
val_root = '../data/source/train_val_sort/val'
checkpoint_prefix = 'models/ckpt_{epoch}'
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [3]:
model = tf.keras.models.Sequential()
model.add(ConvLSTM2D(32, (3,3), strides=(2,2), 
                            padding='valid', 
                            data_format='channels_last',
                            return_sequences=True))
model.add(ConvLSTM2D(32, (3,3), strides=(2,2), 
                            padding='valid', 
                            data_format='channels_last',
                            return_sequences=True,
                            ))
model.add(ConvLSTM2D(32, (3,3), strides=(2,2), 
                            padding='valid', 
                            data_format='channels_last',
                            return_sequences=True,
                            ))
model.add(ConvLSTM2D(32, (3,3), strides=(2,2), 
                            padding='valid', 
                            data_format='channels_last', 
                            return_sequences=False,
                            ))
model.add(Conv2D(16 , (3,3), strides=(2,2), 
                        padding='valid', data_format='channels_last',
                        activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(16 , (3,3), strides=(2,2), 
                        padding='valid', data_format='channels_last',
                        activation='relu'))
model.add(Flatten())
model.add(Dense(1,
                activation='sigmoid'))

In [4]:
model.load_weights('models/model_0304_1000_steps_01_reg/ckpt_9')
model.build((None,None,224,224,3))

In [10]:
val_ds = tf.data.Dataset.list_files(str(pathlib.Path(val_root)/'*/*'))
val_ds = val_ds.shuffle(10000)
val_transformer = DeepFakeTransformer(resize_shape=(224,224), seq_length=30)

In [11]:
val_ds = val_ds.map(lambda x: val_transformer.transform_map(x))
val_ds = val_ds.padded_batch(4, padded_shapes=(None)).prefetch(2)

In [7]:
model.predict(val_ds, steps=2)

array([[0.8483074],
       [0.8483074],
       [0.8483074],
       [0.8483074],
       [0.8483074],
       [0.8483074],
       [0.8483074],
       [0.8483074]], dtype=float32)

In [8]:
model.compile(loss=loss)


In [12]:
model.evaluate(val_ds, steps=10)

10/10 [==============================] - 54s 5s/step - loss: 0.4212


0.42118564

In [16]:
np.random.choice(np.arange(0, 300), 30)

array([ 97,  96, 159, 148, 215, 276, 295,  70, 190, 276, 291, 211,  68,
        50, 147, 162,  88, 102, 110, 200, 198, 238, 141, 146,  34, 220,
        47, 282,  65, 235])